In [1]:
import sys
import statistics 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import sklearn
from sklearn import neighbors, datasets
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler 
from skmultilearn.adapt import MLkNN
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

pd.set_option('display.max_columns', None)
np.set_printoptions(threshold=10)
#np.set_printoptions(threshold=sys.maxsize)

In [13]:
train_data = pd.read_csv("C://Users//Sauraj (Work mode)//Desktop//train_new.csv")
test_data = pd.read_csv("C://Users//Sauraj (Work mode)//Desktop//test_new.csv")

## Preprocess data

In [14]:
train_data = train_data.drop('Unnamed: 0', axis=1)

In [15]:
test_data = test_data.drop('Unnamed: 0', axis=1)

In [16]:
train_data = train_data.drop('label', axis=1)

In [17]:
cols = train_data.columns

In [18]:
train_data[train_data[cols].duplicated()]

,month,weekday,hour_offset,ccs,bcced,images,urls,salutations,designation,chars_in_subject,chars_in_body,total_mails_by_sender,sender_freq_total_period,sender_freq_prev_year,sender_freq_prev_week,sender_freq_prev_month,sender_freq_prev_six_months,img_url_ratio,body_subj_ratio,has_subject,has_body,forward_level,No_sender_reported,update_org,personal_org,promotions_org,forums_org,purchases_org,travel_org,spam_org,social_org,update_tld,personal_tld,promotions_tld,forums_tld,purchases_tld,travel_tld,spam_tld,social_tld,forum_sender,social_sender,promo_sender,update_sender,update_sender_only,promo_sender_only,forum_sender_only,social_sender_only,promo_and_update_sender,sender_depth,sender_images_mean,sender_images_std,sender_images_skew,sender_images_count,sender_images_median,sender_images_<function <lambda> at 0x0000020371574438>,sender_urls_mean,sender_urls_std,sender_urls_skew,sender_urls_count,sender_urls_median,sender_urls_<function <lambda> at 0x0000020371574438>,sender_chars_in_subject_mean,sender_chars_in_subject_std,sender_chars_in_subject_skew,sender_chars_in_subject_count,sender_chars_in_subject_median,sender_chars_in_subject_<function <lambda> at 0x0000020371574438>,sender_chars_in_body_mean,sender_chars_in_body_std,sender_chars_in_body_skew,sender_chars_in_body_count,sender_chars_in_body_median,sender_chars_in_body_<function <lambda> at 0x0000020371574438>,sender_total_mails_by_sender_mean,sender_total_mails_by_sender_std,sender_total_mails_by_sender_skew,sender_total_mails_by_sender_count,sender_total_mails_by_sender_median,sender_total_mails_by_sender_<function <lambda> at 0x0000020371574438>,sender_img_url_ratio_mean,sender_img_url_ratio_std,sender_img_url_ratio_skew,sender_img_url_ratio_count,sender_img_url_ratio_median,sender_img_url_ratio_<function <lambda> at 0x0000020371574438>,sender_body_subj_ratio_mean,sender_body_subj_ratio_std,sender_body_subj_ratio_skew,sender_body_subj_ratio_count,sender_body_subj_ratio_median,sender_body_subj_ratio_<function <lambda> at 0x0000020371574438>,sender_images_mean.1,sender_images_std.1,sender_images_skew.1,sender_images_count.1,sender_images_median.1,sender_images_<function <lambda> at 0x0000020371574438>.1,sender_urls_mean.1,sender_urls_std.1,sender_urls_skew.1,sender_urls_count.1,sender_urls_median.1,sender_urls_<function <lambda> at 0x0000020371574438>.1,sender_chars_in_subject_mean.1,sender_chars_in_subject_std.1,sender_chars_in_subject_skew.1,sender_chars_in_subject_count.1,sender_chars_in_subject_median.1,sender_chars_in_subject_<function <lambda> at 0x0000020371574438>.1,sender_chars_in_body_mean.1,sender_chars_in_body_std.1,sender_chars_in_body_skew.1,sender_chars_in_body_count.1,sender_chars_in_body_median.1,sender_chars_in_body_<function <lambda> at 0x0000020371574438>.1,sender_total_mails_by_sender_mean.1,sender_total_mails_by_sender_std.1,sender_total_mails_by_sender_skew.1,sender_total_mails_by_sender_count.1,sender_total_mails_by_sender_median.1,sender_total_mails_by_sender_<function <lambda> at 0x0000020371574438>.1,sender_img_url_ratio_mean.1,sender_img_url_ratio_std.1,sender_img_url_ratio_skew.1,sender_img_url_ratio_count.1,sender_img_url_ratio_median.1,sender_img_url_ratio_<function <lambda> at 0x0000020371574438>.1,sender_body_subj_ratio_mean.1,sender_body_subj_ratio_std.1,sender_body_subj_ratio_skew.1,sender_body_subj_ratio_count.1,sender_body_subj_ratio_median.1,sender_body_subj_ratio_<function <lambda> at 0x0000020371574438>.1,sender_images_mean.2,sender_images_std.2,sender_images_skew.2,sender_images_count.2,sender_images_median.2,sender_images_<function <lambda> at 0x0000020371574438>.2,sender_urls_mean.2,sender_urls_std.2,sender_urls_skew.2,sender_urls_count.2,sender_urls_median.2,sender_urls_<function <lambda> at 0x0000020371574438>.2,sender_chars_in_subject_mean.2,sender_chars_in_subject_std.2,sender_chars_in_subject_skew.2,sender_chars_in_subject_count.2,sender_chars_in_subject_median.2,sender_chars_in_subject_<function <lambda> at 0x00000203715744

In [141]:
#Fill missing hour offsets in "hour_offset" to -1
train_data['hour_offset'] = train_data['hour_offset'].fillna(0)
train_data['sender_freq_total_period'] = train_data['sender_freq_total_period'].fillna(0)
train_data['sender_freq_prev_year'] = train_data['sender_freq_prev_year'].fillna(0)
train_data['sender_freq_prev_week'] = train_data['sender_freq_prev_week'].fillna(0)
train_data['sender_freq_prev_month'] = train_data['sender_freq_prev_month'].fillna(0)
train_data['sender_freq_prev_six_months'] = train_data['sender_freq_prev_six_months'].fillna(0)
train_data['chars_in_subject'] = train_data['chars_in_subject'].fillna(0)
train_data['img_url_ratio'] = train_data['img_url_ratio'].fillna(0)
train_data['body_subj_ratio'] = train_data['body_subj_ratio'].fillna(0)
train_data['image_mean_value'] = train_data['image_mean_value'].fillna(0)
train_data['image_std_value'] = train_data['image_std_value'].fillna(0)
train_data['urls_mean_value'] = train_data['urls_mean_value'].fillna(0)
train_data['urls_std_value'] = train_data['urls_std_value'].fillna(0)
train_data['chars_in_subject_mean_value'] = train_data['chars_in_subject_mean_value'].fillna(0)
train_data['chars_in_subject_std_value'] = train_data['chars_in_subject_std_value'].fillna(0)
train_data['chars_in_body_mean_value'] = train_data['chars_in_body_mean_value'].fillna(0)
train_data['chars_in_body_std_value'] = train_data['chars_in_body_std_value'].fillna(0)

In [39]:
train_data = train_data.replace([np.inf, -np.inf], 0)
test_data = test_data.replace([np.inf, -np.inf], 0)

train_data = train_data.fillna(0) 
test_data = test_data.fillna(0)

## KNN

In [6]:
n_neighbors = 5

X = train_data.drop(['label'], axis=1)
y = train_data['label']

# scaler = StandardScaler()
# scaled_Xtr = scaler.fit_transform(X, y)

# kf = StratifiedKFold(n_splits=5)
# print(kf)

# print("Neighbours:", n_neighbors)

# f1s_train = []
# f1s_test = []

# for train_index, test_index in kf.split(X, y):
#     #print("TRAIN:", train_index, "TEST:", test_index)
#     X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#     y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
#     # we create an instance of Neighbours Classifier and fit the data.
#     clf = neighbors.KNeighborsClassifier(n_neighbors, algorithm='kd_tree')
#     clf.fit(X_train, y_train)

#     y_train_pred = clf.predict(X_train)
#     y_test_pred = clf.predict(X_test)
    
#     f1_train = sklearn.metrics.f1_score(y_train, y_train_pred, average='micro')
#     f1_test = sklearn.metrics.f1_score(y_test, y_test_pred, average='micro')
    
#     f1s_train.append(f1_train)
#     f1s_test.append(f1_test)
    
#     #print("F1 train:", f1_train)
#     print("F1 test:", f1_test)

# print("mean f1 train:", statistics.mean(f1s_train))
# print("mean f1 test:", statistics.mean(f1s_test))

## Multilabel knn

In [42]:
X = train_data.drop(['label'], axis=1)
duplicate_indices = X.groupby(X.columns.tolist(),as_index=False).indices.values()
y_distinct = np.zeros((len(X), 8))
for i in duplicate_indices:
    #print(i)
    index = min(i)
    #print(index)
    #print(len(X.iloc[i].drop_duplicates()))
    labels = train_data.loc[i]['label'].values
    #print(labels)
    for l in labels:
        y_distinct[index,l] = 1

In [60]:
y_distinct

array([[0., 0., 1., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [44]:
y_distinct = y_distinct[~np.all(y_distinct == 0, axis=1)]
X_distinct = X.drop_duplicates().reset_index()

In [46]:
X_distinct = X_distinct.drop(['index'], axis=1)

In [28]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_Xtr = scaler.fit_transform(X_distinct)

In [29]:
print(len(y_distinct),len(X_distinct))

47981 47981


In [ ]:
kf = KFold(n_splits=5)

for train_index, test_index in kf.split(scaled_Xtr, y_distinct):
    # split k-folds
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = scaled_Xtr[train_index], scaled_Xtr[test_index]
    y_train, y_test = y_distinct[train_index], y_distinct[test_index]
    
    # train
    classifier = MLkNN(k=6)
    classifier.fit(X_train, y_train)
    
    # predict
    y_pred = classifier.predict(X_test)
    print(y_pred.shape, y_test.shape)
    
    # measure
    f1 = sklearn.metrics.f1_score(y_test, y_pred, average='micro')
    print(f1)
    
    #print(predictions)

TRAIN: [ 9597  9598  9599 ... 47978 47979 47980] TEST: [   0    1    2 ... 9594 9595 9596]


C:\Users\Sauraj (Work mode)\Miniconda3\lib\site-packages\sklearn\utils\validation.py:71: FutureWarning: Pass n_neighbors=6 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)
